In [ ]:
from googleapiclient.discovery import build
import json, time, sys
from collections import deque
import pandas as pd

video_collected = []

def formatComment(comment):
    doc = {}
    doc['id']=comment['id']
    doc['comment'] = comment['snippet']['topLevelComment']['snippet']['textOriginal']
    doc['user'] = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
    try:
        doc['userId'] = comment['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
    except:
        doc['userId'] = ""
    doc['likeCount'] = comment['snippet']['topLevelComment']['snippet']['likeCount']
    doc['publishedAt']=comment['snippet']['topLevelComment']['snippet']['publishedAt']
    doc['totalReplyCount'] = comment['snippet']['totalReplyCount']
    doc['videoId'] = comment['snippet']['videoId']
    doc['parentId'] = ""
    return doc

def formatReply(reply):
    doc = {}
    doc['id']=reply['id']
    doc['comment'] = reply['snippet']['textOriginal']
    doc['user'] = reply['snippet']['authorDisplayName']
    try:
        doc['userId'] = reply['snippet']['authorChannelId']['value']
    except:
        doc['userId'] = ""
    doc['likeCount'] = reply['snippet']['likeCount']
    doc['publishedAt']=reply['snippet']['publishedAt']
    doc['totalReplyCount'] = 0
    doc['videoId'] = reply['snippet']['videoId']
    doc['parentId'] = reply['snippet']['parentId']
    return doc

class Collector:
    def __init__(self,collected_videos=set()):
        with open('config.json') as config:
            config=json.load(config)
            keys = config['api_keys']
        self.youtubes = [build('youtube', 'v3', developerKey=key) for key in keys]
        self.youtube = self.youtubes[0]
        self.current_queries = 0
        self.collected_videos = collected_videos
        
        
    def roll(self):
        try:
            self.youtube = self.youtubes[self.current_queries//9000]
        except:
            print('Quota exceeded,sleeping')
            time.sleep(60*60*24)
            self.youtube=self.youtubes[0]
            self.current_queries = 0
    
    #function to get the top videos for each category according to a sorting method
    def getVideos(self, categories,method='relevance'):
        videos_metadata=[]
        part=["id","snippet"]
        for category in categories:
            videos = []

            while True:
                try:
                    resp = self.youtube.search().list(regionCode='us',part='snippet',maxResults=50,type='video'
                                                    ,order=method,relevanceLanguage='en',videoCategoryId=category)
                    result = resp.execute()
                    self.current_queries+=100
                    self.roll()
                    break
                except:#service unavailable
                    time.sleep(60)
            for item in result['items']:
                if item['id']['videoId'] in self.collected_videos:
                    continue
                formattedResult = {}
                formattedResult['videoId'] = item['id']['videoId']
                formattedResult['publishedAt']=item['snippet']['publishedAt']
                formattedResult['title']=item['snippet']['title']
                formattedResult['description']=item['snippet']['description']
                formattedResult['channelTitle']=item['snippet']['channelTitle']
                formattedResult['channelId']=item['snippet']['channelId']
                formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                formattedResult['category'] = category
                formattedResult['method'] = method
                videos.append(formattedResult)
                self.collected_videos.add(item['videoId'])
            while result:
                if (len(videos) >= 1000) or ('nextPageToken' not in result):
                    break
                while True:
                    try:
                        resp = self.youtube.search().list(regionCode='us',pageToken=result['nextPageToken'],part='snippet',maxResults=50,type='video'
                                                    ,order=method,relevanceLanguage='en',videoCategoryId=category)
                        result = resp.execute()
                        self.current_queries+=100
                        self.roll()
                        break
                    except:
                        time.sleep(60)
                for item in result['items']:
                    if item['id']['videoId'] in self.collected_videos:
                        continue
                    formattedResult = {}
                    formattedResult['videoId'] = item['id']['videoId']
                    formattedResult['publishedAt']=item['snippet']['publishedAt']
                    formattedResult['title']=item['snippet']['title']
                    formattedResult['description']=item['snippet']['description']
                    formattedResult['channelTitle']=item['snippet']['channelTitle']
                    formattedResult['channelId']=item['snippet']['channelId']
                    formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                    formattedResult['category'] = category
                    formattedResult['method'] = method 
                    videos.append(formattedResult)
                    self.collected_videos.add(item['videoId'])
            videos_metadata.extend(videos)
        return videos_metadata
    
    #function to get video metadata and view/like/comment counts
    def getVideoStatistics(self,videos):
        for video in videos:
            print(video['videoId'])
            while True:
                try:
                    resp = self.youtube.videos().list(part=['snippet','contentDetails','topicDetails','statistics'],id=video['videoId'])
                    result = resp.execute()
                    self.current_queries+=1
                    self.roll()
                    break
                except Exception as e:
                    print(e)
                    time.sleep(60)
            for item in result['items']:
                try:
                    video['tags'] = item['snippet']['tags']
                except:
                    video['tags'] = []
                try:
                    video['language'] = item['snippet']['defaultLanguage']
                except:
                    video['language'] = ''
                video['duration'] = item['contentDetails']['duration']
                try:
                    video['topics'] = item['topicDetails']['topicCategories']
                except:
                    video['topics'] = []
                video.update(item['statistics'])
        return videos
    
    #function to get channel metadata
    def getChannelStatistics(self,channels):
        channels_metadata=[]
        for channelId in channels:
            while True:
                try:
                    resp= self.youtube.channels().list(part=['id','statistics','snippet'],id=channelId)
                    result = resp.execute()
                    self.current_queries+=1
                    self.roll()
                    break
                except Exception as e:
                    raise(e)
                    time.sleep(60)
            if 'items' not in result:
                continue
            channel_data = {}
            channel_data['title']=result['items'][0]['snippet']['title']
            channel_data['id'] = result['items'][0]['id']
            channel_data['viewCount'] = result['items'][0]['statistics']['viewCount']
            try:
                channel_data['subscriberCount'] = result['items'][0]['statistics']['subscriberCount']
            except:
                channel_data['subscriberCount'] = None
            try:
                channel_data['description'] = result['items'][0]['snippet']['description']
            except:
                channel_data['description'] = ''
            try:
                channel_data['publishedAt'] = result['items'][0]['snippet']['publishedAt']
            except:
                channel_data['publishedAt'] = None
            channel_data['videoCount'] = result['items'][0]['statistics']['videoCount']
            channels_metadata.append(channel_data)
        return channels_metadata

    #function to backup currently collected videos
    def backup(self):
        with open('Data/temp_videos.jsonl','a',encoding='utf-8') as file:
            for video in self.videos_metadata:
                json.dump(video,file,ensure_ascii=False)
                file.write('\n')
        with open('Data/backup.txt','w') as backup:
            for pageToken,channel in self.page_queue:
                backup.write(str(pageToken)+','+str(channel))
                backup.write('\n')
        self.videos_metadata = []
        self.n_iter=0
        
    #gets top 50 videos from a channel
    def getFromChannel(self,channels,iter_to_backup=10):
        self.videos_metadata=[]
        part=["id","snippet"]
        self.n_iter=0
        for channel in channels:
            self.n_iter+=1
            if self.n_iter%iter_to_backup == 0:
                print(f'backing up data at {self.n_iter} iterations')
                self.backup()
            videos = []
            while True:
                try:
                    resp = self.youtube.search().list(regionCode='us',part='snippet',maxResults=50,type='video'
                                            ,order='viewCount',relevanceLanguage='en',channelId=channel['id'])
                    result = resp.execute()
                    self.current_queries+=100
                    self.roll()
                    break
                except Exception as e:#service unavailable
                    print(e)
                    time.sleep(60)
            for item in result['items']:
                formattedResult = {}
                formattedResult['videoId'] = item['id']['videoId']
                formattedResult['publishedAt']=item['snippet']['publishedAt']
                formattedResult['title']=item['snippet']['title']
                formattedResult['description']=item['snippet']['description']
                formattedResult['channelTitle']=item['snippet']['channelTitle']
                formattedResult['channelId']=item['snippet']['channelId']
                formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                formattedResult['category'] = item['snippet']['categoryId']
                formattedResult['method'] = 'from_channel'
                videos.append(formattedResult)
            if result and 'nextPageToken' in result:
                self.page_queue.append((result['nextPageToken'],channel['id']))
            self.videos_metadata.extend(videos)

        self.backup()
        
        return True
    
    #gets up to "limit" comments for a given video
    def getComments(self,videoId,limit=float('inf')):
        comments = []
        response = self.youtube.commentThreads().list(part=["id","snippet","replies"],videoId=videoId,maxResults=100).execute()
        self.current_queries+=1
        self.roll()
        while response:
            for item in response['items']:
                formattedResult = formatComment(item)
                comments.append(formattedResult)
                if len(comments)%10000 ==0:
                    print(f'{len(comments)} collected for {videoId}')
                if len(comments)>=limit:
                    return comments
                if formattedResult['totalReplyCount']>0:
                    try:
                        for reply in item['replies']['comments']:
                            reply = formatReply(reply)
                            comments.append(reply)
                    except:
                        pass
            if 'nextPageToken' in response:
                while True:
                    try:
                        response = self.youtube.commentThreads().list(part=["id","snippet","replies"],videoId = videoId,maxResults=100,pageToken=response['nextPageToken']).execute()
                        self.current_queries+=1
                        self.roll()
                        break
                    except:
                        time.sleep(60)
            else:
                break
        return comments

# Main collection loop - Part 1

In [ ]:
try:
    with open('Data/categories.json') as file:
        categories = json.load(file)
except:
    categories = youtube.videoCategories().list(part='snippet',regionCode='US')
    with open('Data/categories.json','w') as file:
        json.dump(categories,file)


category_ids = [category['id'] for category in categories]
print(f'Collecting data for {len(category_ids)} categories using method')

collector = Collector(collected_videos=set())
videos=collector.getVideos(category_ids,method='viewCount')
print(f'Finished category queries {len(videos)} found')
videos = collector.getVideoStatistics(videos)
print(f'Finished collecting statistics')
with open('Data/video_metadata.jsonl','a',encoding='utf-8') as file:
    for line in videos:
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [ ]:
for video in videos:
    video_collected.append(video['videoId'])
    try:
        comments=collector.getComments(video['videoId'])
    except Exception as e:
        if 'disabled comments' in str(e):
            print(f'Video {video["videoId"]} has disabled comments')
            continue
    print(f'Collected {len(comments)} comments for {video["videoId"]}')
    with open('Data/comments.jsonl','a',encoding='utf-8') as file:
        for comment in comments:
            json.dump(comment,file,ensure_ascii=False)
            file.write('\n')
print('Done')

In [ ]:
channels = set([video['channelId'] for video in videos])
print(f'Collecting data for {len(channels)} channels')
channel_statistics = collector.getChannelStatistics(channels)
with open('Data/channel_metadata.jsonl','w',encoding='utf-8') as file:
     for channel in channel_statistics:
        json.dump(channel,file,ensure_ascii=False)
        file.write('\n')

# Collection loop - part 2 
## Popular videos from collected channels

In [ ]:
channels=pd.read_csv('channels_relevant.csv').to_dict(orient='records')

In [ ]:
collector.getFromChannel(channels,iter_to_backup=1)

In [ ]:
#keep only top 3 videos due to time_constraints
videos = pd.read_json('Data/temp_videos.jsonl',lines=True)
vid = pd.read_json('Data/video_metadata.jsonl',lines=True)
videos_to_collect = []
for c,df in videos.groupby('channelId'):
    df = df[~df['videoId'].isin(vid['videoId'].tolist())]
    df = df[:3]
    videos_to_collect.append(df.copy())
videos_to_collect = pd.concat(videos_to_collect).reset_index(drop=True)

In [ ]:
videos_to_collect = videos_to_collect.to_dict(orient='records')
video_metadata = collector.getVideoStatistics(videos_to_collect)

In [ ]:
with open('Data/video_metadata_from_channels.jsonl','w',encoding='utf-8') as file:
    for line in video_metadata:
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [ ]:
#get comments from videos from channels (keep at most 10**5 comments for each video due to time constraints)
for video in videos_to_collect['videoId']:
    if video in video_collected:
        continue
    try:
        comments=collector.getComments(video)
        while comments=='fail':
            print('failed once')
            comments=collector.getComments(video,limit=10**5)
        video_collected.add(video)
    except Exception as e:
        if 'disabled comments' in str(e):
            print(f'Video {video} has disabled comments')
            video_collected.add(video)
            continue
        else:
            print(e)
    print(f'Collected {len(comments)} comments for {video}')
    with open('Data/comments_from_channels.jsonl','a',encoding='utf-8') as file:
        for comment in comments:
            json.dump(comment,file,ensure_ascii=False)
            file.write('\n')